In [68]:
from itertools import chain
from lnn import Predicate, Variable, Exists, Implies, And, ForAll, Model, Fact, World

## Domain specification

In [69]:
# entities and properties names
entities = ["rectangle", "square", "rhombus", "parallelogram"]
properties = ["foursided", "equal_sides", "right_angles"]

# implications
entity_relationships = [
    ("square", "rectangle"),
    ("square", "rhombus"),
    ("rectangle", "parallelogram"),
    ("rhombus", "parallelogram"),
]

# implications
entity_properties = [
    ("parallelogram", "foursided"),
    ("rectangle", "right_angles"),
    ("rhombus", "equal_sides"),
]

# information that object "s1" is a square etc.
facts = {
    "square": ["s1", "s2"],
    "rectangle": ["r1"],
    "rhombus": ["rh1"],
    "parallelogram": ["p1"],
}

## Model definition and domain parsing

In [72]:
model = Model()

x = Variable("x")

# parsing predicates
for predicate in chain(entities, properties):
    model[predicate] = Predicate(name=predicate)

model.print()


***************************************************************************
                                LNN Model

OPEN   Predicate: right_angles(x0) 

OPEN   Predicate: equal_sides(x0) 

OPEN   Predicate: foursided(x0) 

OPEN   Predicate: parallelogram(x0) 

OPEN   Predicate: rhombus(x0) 

OPEN   Predicate: square(x0) 

OPEN   Predicate: rectangle(x0) 

***************************************************************************


In [73]:
# parsing axioms in Implication and ForAll shape
axioms = [
    ForAll(
        x,
        Implies(model[p1](x), model[p2](x), name=f"{p1}_{p2}"),
        name=f"all_{p1}_{p2}",
        world=World.AXIOM,
    )
    for p1, p2 in chain(entity_relationships, entity_properties)
]

model.add_formulae(*axioms)

model.print()


***************************************************************************
                                LNN Model

AXIOM  ForAll: all_rhombus_equal_sides (x)                  TRUE (1.0, 1.0)

OPEN   Implies: rhombus_equal_sides(x) 

AXIOM  ForAll: all_rectangle_right_angles (x)               TRUE (1.0, 1.0)

OPEN   Implies: rectangle_right_angles(x) 

AXIOM  ForAll: all_parallelogram_foursided (x)              TRUE (1.0, 1.0)

OPEN   Implies: parallelogram_foursided(x) 

AXIOM  ForAll: all_rhombus_parallelogram (x)                TRUE (1.0, 1.0)

OPEN   Implies: rhombus_parallelogram(x) 

AXIOM  ForAll: all_rectangle_parallelogram (x)              TRUE (1.0, 1.0)

OPEN   Implies: rectangle_parallelogram(x) 

AXIOM  ForAll: all_square_rhombus (x)                       TRUE (1.0, 1.0)

OPEN   Implies: square_rhombus(x) 

AXIOM  ForAll: all_square_rectangle (x)                     TRUE (1.0, 1.0)

OPEN   Implies: square_rectangle(x) 

OPEN   Predicate: right_angles(x0) 

OPEN   Predi

In [65]:
# parsing facts
model.add_facts(
    {entity: {name: Fact.TRUE for name in names} for entity, names in facts.items()}
)

## Question definition

In [74]:
queries = [
    Exists(x, model["foursided"](x), name="foursided_objects"),
    Exists(x, model["equal_sides"](x), name="equal_sides_objects"),
    Exists(x, model["right_angles"](x), name="right_angles_objects"),
    Exists(
        x,
        And(model["right_angles"](x), model["equal_sides"](x)),
        name="right_angles_and_equal_length_sides",
    ),
]

model.add_formulae(*queries)

model.print()


***************************************************************************
                                LNN Model

OPEN   Exists: right_angles_and_equal_length_sides (x)       UNKNOWN (0.0, 1.0)

OPEN   And: And_11(x) 

OPEN   Exists: right_angles_objects (x)                  UNKNOWN (0.0, 1.0)

OPEN   Exists: equal_sides_objects (x)                   UNKNOWN (0.0, 1.0)

OPEN   Exists: foursided_objects (x)                     UNKNOWN (0.0, 1.0)

AXIOM  ForAll: all_rhombus_equal_sides (x)                  TRUE (1.0, 1.0)

OPEN   Implies: rhombus_equal_sides(x) 

AXIOM  ForAll: all_rectangle_right_angles (x)               TRUE (1.0, 1.0)

OPEN   Implies: rectangle_right_angles(x) 

AXIOM  ForAll: all_parallelogram_foursided (x)              TRUE (1.0, 1.0)

OPEN   Implies: parallelogram_foursided(x) 

AXIOM  ForAll: all_rhombus_parallelogram (x)                TRUE (1.0, 1.0)

OPEN   Implies: rhombus_parallelogram(x) 

AXIOM  ForAll: all_rectangle_parallelogram (x)              TRU

## Infer from model and answer questions

In [67]:
steps, facts_inferred = model.infer()

for query in queries:
    print(f"Query {query.name} result:")
    print(model[query.name].true_groundings)

Query foursided_objects result:
{'p1', 'r1', 'rh1', 's1', 's2'}
Query equal_sides_objects result:
{'s2', 'rh1', 's1'}
Query right_angles_objects result:
{'s2', 'r1', 's1'}
Query right_angles_and_equal_length_sides result:
{'s2', 's1'}
